In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import wandb

from typing import Union

In [ ]:
def wandb2pd(exp_runs):
    df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    summary_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    config_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    name_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    summary = [] 
    config = [] 
    name = [] 
    for exp in exp_runs: 
        summary.append(exp.summary._json_dict) 
        config.append({k:v for k,v in exp.config.items() if not k.startswith('_')}) 
        name.append(exp.name)       

    summary_df = pd.DataFrame.from_records(summary) 
    config_df = pd.DataFrame.from_records(config) 
    name_df = pd.DataFrame({'name': name}) 
    df = pd.concat([name_df, config_df, summary_df], axis=1)
    return df


In [ ]:
def plot_status_by_trial_budget(
        df             :pd.DataFrame,
        plot_column    :str,
        top_k          :Union[int, str],
        optimizer_name :str,
        in_or_out      :str,
        max_xlim       :int,
        max_ylim       :float,
        is_log_xscale  :bool=False,
    ):
    plt.figure(figsize=(6,4))
    plt.get_cmap('Blues') 
    
    ns = [n*10 for n in range(max_xlim)]
    
    acc_maxes = [df.head(n)[plot_column].max() for n in ns]
    plt.plot(ns, acc_maxes, label="top-1")
    
    acc_averages = [
        df.head(n)\
          .sort_values(by=plot_column, axis=0, ascending=False)[plot_column]\
          .head(top_k)\
          .mean()\
        for n in ns
    ]
    plt.plot(ns, acc_averages, label=f"top-{top_k} average")
    
    acc_averages = [
        df.head(n)[plot_column]\
          .mean()\
        for n in ns
    ]
    plt.plot(ns, acc_averages, label=f"all average")

    plt.title(f"Background Challenge {optimizer_name}", fontsize=12)
    plt.xlabel(f"trials", fontsize=14)
    plt.ylabel(f"{in_or_out}-distribution acc", fontsize=14)
    plt.ylim(top=max_ylim+max_ylim*0.01)
    
    if is_log_xscale:
        plt.xscale("log")
        
    plt.grid(linewidth=1, alpha=0.5)
    plt.legend()
    
    if is_log_xscale:
        plt.savefig(f"bgc/figs/accplot_by_trial_budget/bgc_{in_or_out}dist_acc_{optimizer_name}_logscale.pdf")
    else:
        plt.savefig(f"bgc/figs/accplot_by_trial_budget/bgc_{in_or_out}dist_acc_{optimizer_name}.pdf")
    
    return 

In [ ]:
exp_df_dict = {}
optimizer_name_list = [
    "Momentum",
    "Adam"
]
path_list = [
    f"entity_name/project_momentum_sgd",
    f"entity_name/project_adam"
]
api = wandb.Api()

In [ ]:
for i in range(len(path_list)):
    path = path_list[i]
    optimizer_name = optimizer_name_list[i]
    
    exp_runs = api.runs(
        path=path,
        filters={"state":"finished"}
    )

    df = wandb2pd(exp_runs)
    df.sort_values(by="_timestamp", axis=0, ascending=True, inplace=True)
    
    exp_df_dict[optimizer_name] = df

In [ ]:
min_exp_length = 10**5
max_in_acc = 0
max_out_acc = 0

for exp_df in exp_df_dict.values():
    if len(exp_df) < min_exp_length:
        min_exp_length = len(exp_df)
    
    if exp_df["acc_orig"].max() > max_in_acc:
        max_in_acc = exp_df["acc_orig"].max()
        
    if exp_df["acc_mixed_rand"].max() > max_out_acc:
        max_out_acc = exp_df["acc_mixed_rand"].max()
    
print(min_exp_length, max_in_acc, max_out_acc)

### not log scale

In [ ]:
plot_column_list = ["acc_orig", "acc_mixed_rand"]
in_or_out_list = ["in", "out"]
max_ylim_list = [max_in_acc, max_out_acc]

for optimizer_name, exp_df in exp_df_dict.items():
    for plot_column, in_or_out, max_ylim in zip(plot_column_list, in_or_out_list, max_ylim_list):

        plot_status_by_trial_budget(
            df=exp_df,
            plot_column=plot_column,
            top_k=10,
            optimizer_name=optimizer_name,
            in_or_out=in_or_out,
            max_xlim=min_exp_length//10,
            max_ylim=max_ylim,
            is_log_xscale=False,
        )
        

### log scale

In [ ]:
plot_column_list = ["acc_orig", "acc_mixed_rand"]
in_or_out_list = ["in", "out"]
max_ylim_list = [max_in_acc, max_out_acc]

for optimizer_name, exp_df in exp_df_dict.items():
    for plot_column, in_or_out, max_ylim in zip(plot_column_list, in_or_out_list, max_ylim_list):

        plot_status_by_trial_budget(
            df=exp_df,
            plot_column=plot_column,
            top_k=10,
            optimizer_name=optimizer_name,
            in_or_out=in_or_out,
            max_xlim=min_exp_length//10,
            max_ylim=max_ylim,
            is_log_xscale=True,
        )
        